In [ ]:
# Import necessary libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
from crud import AnimalShelter  # Make sure to import your CRUD module

# Initialize your dash application
app = JupyterDash('Dashboard')

# Setup connection to MongoDB
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)  # Adjust as necessary for your CRUD class

# Retrieve data from MongoDB
df = pd.DataFrame(list(shelter.read({})))  # Assuming read method returns all documents
df.drop(columns=['_id'], inplace=True)  # Remove MongoDB's _id column

# Define the layout for your dashboard
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard: William Tunstall'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single",  # Ensure single row selection for mapping callback
        # Add additional features below as needed
    ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6')
])

# Callback for updating the data table styles upon selection
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Your style update code here
    pass
# Function to update the map based on the data table selection
def update_map(viewData, index):
    if index is None or not viewData:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10)]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row,9])])
                      ])
        ])
    ]

# Callback for the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def display_map(viewData, index):
    return update_map(viewData, index)

# Run the server
app.run_server(debug=True)
